In [287]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoost, CatBoostRegressor
import csv
import os
from hyperopt import hp, STATUS_OK, tpe, fmin, Trials
from hyperopt.pyll.stochastic import sample
from timeit import default_timer as timer

In [213]:
customers_tr = pd.read_csv('data_like/customer_train.csv')
stories = pd.read_csv('data_like/stories_description.csv')
reaction = pd.read_csv('data_like/stories_reaction_train.csv')
transactions = pd.read_csv('data_like/transactions.csv')
customers_te = pd.read_csv('data_like/customer_test.csv')
reaction_te = pd.read_csv('data_like/stories_reaction_test.csv')
transactions = pd.read_csv('data_like/transactions.csv')

In [27]:
def mcc_to_category(x):
        if(x in list(range(3000,3300)) + [ 4304, 4415, 4418, 4511, 4582]):
        return 'Авиабилеты'
    elif(x in [5511, 5521, 5531, 5532, 5533, 5571, 7012, 7531, 7534, 7535, 7538, 7542, 7549]):
        return 'Автоуслуги'
    elif(x in list(range(3351,3399)) + list(range(3400,3411)) + list(range(3412,3424)) + list(range(3425,3440)) + [ 3441, 7512, 7513, 7519]):
        return 'Аренда авто'
    elif(x in [5122, 5292, 5295, 5912]):
        return 'Аптеки'
    elif(x in [1520, 1711, 1731, 1740,1750, 1761, 1771, 1799, 2791, 2842, 5021, 5039, 5046, 5051, 5065, 5072, 5074, 5085, 5198, 5200, 5211, 5231, 5251, 5261, 5415, 5712, 5713, 5714, 5718, 5719, 5722, 7622, 7623, 7629, 7641, 7692, 7699]):
        return 'Дом, Ремонт'
    elif(x in [4011, 4112]):
        return 'Ж/д билеты'
    elif(x in [5995]):
        return 'Животные'
    elif(x in [5932, 5937, 5970, 5971, 5972,5973]):
        return 'Искусство'
    elif(x in [7829, 7832, 7841]):
        return 'Кино'
    elif(x in [2741, 5111, 5192, 5942, 5994]):
        return 'Книги'
    elif(x in [5977,7230,7297,7298]):
        return 'Красота'
    elif(x in [5733, 5735]):
        return 'Музыка'
    elif(x in [5094, 5137, 5139, 5611, 5621, 5631, 5641, 5651, 5661, 5681, 5691, 5697, 5698, 5699, 5931, 5944, 5949, 5950, 7296, 7631
]):
        return 'Одежда, Обувь, Ювелирные изделия и часы'
    elif(x in [7911, 7922, 7929, 7932, 7933, 7941, 7991, 7992, 7993, 7994, 7996, 7997, 7998, 7999, 8664]):
        return 'Развлечения'
    elif(x in [5811, 5812, 5813]):
        return 'Рестораны'
    elif(x in [5655, 5940, 5941]):
        return 'Спорттовары'
    elif(x in [5947]):
        return 'Сувениры'
    elif(x in [5297, 5298, 5300, 5411, 5412, 5422, 5441, 5451, 5462, 5499, 5715, 5921]):
        return 'Супермаркеты'
    elif(x in [5172, 5541, 5542, 5983]):
        return 'Топливо'
    elif(x in [4111, 4121, 4131, 4457, 4468, 4784, 4789, 5013, 5271, 5551, 5561, 5592, 5598, 5599, 7511, 7523]):
        return 'Транспорт'
    elif(x in [5814]):
        return 'Фаст фуд'
    elif(x in [5044, 5045, 5946, 7332, 7333, 7338, 7339, 7395]):
        return 'Фото, Видео'
    elif(x in [5193, 5992]):
        return 'Цветы'
    elif(x in [5309]):
        return 'Duty Free'
    
    

In [126]:
transactions['merchant_mcc'] = transactions['merchant_mcc'].apply(mcc_to_category)
transactions['merchant_mcc'].fillna('Другое', inplace=True)

In [109]:
all_transactions = transactions.groupby(['customer_id', 'merchant_mcc'])['transaction_amt'].sum()

In [110]:
all_transactions = all_transactions.unstack(level=-1).reset_index()
all_transactions = all_transactions.set_index('customer_id', drop=True)

In [120]:
customers_tr.set_index('customer_id', inplace = True)
customers_te.set_index('customer_id', inplace = True)

In [122]:
customers_tr = customers_tr.join(all_transactions)
customers_te = customers_te.join(all_transactions)

In [127]:
# all_transactions = transactions.groupby(['customer_id', 'merchant_mcc'])['transaction_amt'].count()
# all_transactions = all_transactions.unstack(level=-1).reset_index()
# all_transactions = all_transactions.set_index('customer_id', drop=True)

In [197]:
stories['name'] = stories['story_json'].apply(lambda x: ','.join(re.findall(r'"name":"([^"]+)"', x)))

In [200]:
stories['text'] = stories['story_json'].apply(lambda x: ','.join(re.findall(r'"text":"([^"]+)"', x)))

,product_0,product_1,product_2,product_3,product_4,product_5,product_6,gender_cd,age,marital_status_cd,...,Развлечения,Рестораны,Спорттовары,Сувениры,Супермаркеты,Топливо,Транспорт,Фаст фуд,"Фото, Видео",Цветы
customer_id,,,,,,,,,,,,,,,,,,,,,
894436,NaN,NaN,NaN,NaN,NaN,UTL,NaN,M,30.0,MAR,...,NaN,NaN,2500.0,NaN,8000.0,1500.0,NaN,500.0,NaN,NaN
524526,NaN,UTL,NaN,NaN,NaN,UTL,NaN,F,20.0,UNM,...,1500.0,7500.0,10500.0,NaN,14000.0,NaN,1000.0,1500.0,NaN,NaN
498134,NaN,UTL,NaN,NaN,NaN,NaN,NaN,F,25.0,UNM,...,NaN,NaN,NaN,NaN,35500.0,16000.0,NaN,11500.0,NaN,NaN
278941,NaN,NaN,UTL,CLS,NaN,UTL,UTL,M,25.0,NaN,...,6000.0,30500.0,NaN,NaN,46500.0,16000.0,8000.0,37000.0,1500.0,3500.0
877312,NaN,UTL,NaN,NaN,NaN,NaN,NaN,F,40.0,MAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821806,NaN,NaN,NaN,UTL,NaN,UTL,NaN,F,25.0,NaN,...,NaN,1000.0,2000.0,0.0,6500.0,NaN,2500.0,2500.0,NaN,NaN
782728,NaN,UTL,NaN,NaN,NaN,NaN,NaN,M,35.0,MAR,...,NaN,1500.0,1500.0,NaN,17500.0,NaN,NaN,NaN,NaN,NaN
540071,NaN,NaN,NaN,NaN,NaN,UTL,NaN,M,30.0,NaN,...,NaN,500.0,NaN,NaN,5000.0,NaN,0.0,4500.0,NaN,NaN
66158,NaN,UTL,NaN,NaN,NaN,UTL,NaN,F,30.0,MAR,...,NaN,1000.0,NaN,NaN,4500.0,NaN,NaN,0.0,NaN,NaN


In [214]:
reaction['event_dttm'] = pd.to_datetime(reaction['event_dttm'])
reaction.set_index('event_dttm', inplace=True)
reaction['Morning'] = (reaction.index.hour<12).astype('int')
reaction['Evening'] = (reaction.index.hour>18).astype('int')
reaction['Midday'] = np.all([reaction.index.hour<=18, reaction.index.hour>=12], axis=0).astype('int')
reaction_te['event_dttm'] = pd.to_datetime(reaction_te['event_dttm'])
reaction_te.set_index('event_dttm', inplace=True)
reaction_te['Morning'] = (reaction_te.index.hour<12).astype('int')
reaction_te['Evening'] = (reaction_te.index.hour>18).astype('int')
reaction_te['Midday'] = np.all([reaction_te.index.hour<=18, reaction_te.index.hour>=12], axis=0).astype('int')

In [218]:
reaction.reset_index(inplace=True)
reaction_te.reset_index(inplace=True)

In [223]:
reaction = reaction.set_index('customer_id').join(customers_tr)
reaction_te = reaction_te.set_index('customer_id').join(customers_te)

In [254]:
reaction['life_time'] = (reaction['event_dttm'] - pd.to_datetime(reaction['first_session_dttm'])).astype('timedelta64[D]').fillna(-1).astype('int')
reaction_te['life_time'] = (reaction_te['event_dttm'] - pd.to_datetime(reaction_te['first_session_dttm'])).astype('timedelta64[D]').fillna(-1).astype('int')

In [256]:
reaction.isnull().sum()

event_dttm                                      0
story_id                                        0
event                                           0
Morning                                         0
Evening                                         0
Midday                                          0
product_0                                  460949
product_1                                  327250
product_2                                  381085
product_3                                  352388
product_4                                  450820
product_5                                   98814
product_6                                  437861
gender_cd                                   38291
age                                         38071
marital_status_cd                          259536
children_cnt                                87844
first_session_dttm                          36631
job_position_cd                             36597
job_title                                  257797


In [255]:
all_transactions.columns

Index(['Duty Free', 'Авиабилеты', 'Автоуслуги', 'Аптеки', 'Аренда авто',
       'Дом, Ремонт', 'Другое', 'Ж/д билеты', 'Животные', 'Искусство', 'Кино',
       'Книги', 'Красота', 'Музыка', 'Одежда, Обувь, Ювелирные изделия и часы',
       'Развлечения', 'Рестораны', 'Спорттовары', 'Сувениры', 'Супермаркеты',
       'Топливо', 'Транспорт', 'Фаст фуд', 'Фото, Видео', 'Цветы'],
      dtype='object', name='merchant_mcc')

In [258]:
target = 'event'
cat_features = ['story_id', 'job_position_cd', 'marital_status_cd', 'gender_cd', 'Midday', 'Evening', 'Morning'] + ['product_{}'.format(i) for i in range(6)]
num_features = list(all_transactions.columns) + ['age', 'children_cnt', 'life_time']

In [272]:
X_train = reaction[cat_features+num_features]
y_train = reaction[target]
X_test = reaction_te[cat_features+num_features]


In [306]:
y_res = reaction[target]

In [276]:
X_train[cat_features] = X_train[cat_features].fillna('Na')
X_train[num_features] = X_train[num_features].fillna(-1)
X_test[cat_features] = X_test[cat_features].fillna('Na')
X_test[num_features] = X_test[num_features].fillna(-1)

/Users/valyagorbunova/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [277]:
X_train.isnull().sum()

story_id                                   0
job_position_cd                            0
marital_status_cd                          0
gender_cd                                  0
Midday                                     0
Evening                                    0
Morning                                    0
product_0                                  0
product_1                                  0
product_2                                  0
product_3                                  0
product_4                                  0
product_5                                  0
Duty Free                                  0
Авиабилеты                                 0
Автоуслуги                                 0
Аптеки                                     0
Аренда авто                                0
Дом, Ремонт                                0
Другое                                     0
Ж/д билеты                                 0
Животные                                   0
Искусство 

In [293]:
y_train.unique()

array(['view', 'skip', 'like', 'dislike'], dtype=object)

In [298]:
d = {'view':0.2, 'skip':-0.2, 'like':1, 'dislike':-1}
y_train = y_train.apply(lambda x: d[x])

In [307]:
l = {'view':0.1, 'skip':-0.1, 'like':0.5, 'dislike':-10}
y_res = y_res.apply(lambda x: l[x])

In [309]:
def metr(y_pred):
    return sum(y_pred * y_res)

In [310]:
metr(y_train)

102291.08000022317

In [313]:
from hyperopt import fmin, tpe, hp
import csv
from catboost import CatBoostRegressor
import hyperopt
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import KFold 
def cat_cv(alg, X_train, y_train,cat_feat_pos , n=3):
    skf = KFold(n_splits=n, random_state=42, shuffle=True )
    errors = []
    for train_index, test_index in skf.split(X_train, y_train):
        train, test = X_train.iloc[train_index], X_train.iloc[test_index]
        target_train, target_test = y_train.iloc[train_index], y_train.iloc[test_index]
        alg.fit(train, target_train, eval_set=(test, target_test),cat_features=cat_feat_pos)
        error = metr(target_test) - metr(alg.predict(test))
#         error = alg.get_best_score()['learn']['Logloss']
        n_trees = alg.get_best_iteration()
        errors.append(error)
    return np.mean(np.array(errors)), n_trees



global ITERATION
def objective(params, early_stopping_rounds=50):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""

    # Keep track of evals
    global ITERATION
    ITERATION += 1

    # Make sure parameters that need to be integers are integers
    for parameter_name in ['max_depth']:
        params[parameter_name] = int(params[parameter_name])

    start = timer()
    # Perform n_folds cross validation
    cat = CatBoostRegressor(iterations=5000, verbose=0,
                        has_time=True, use_best_model=True, **params)

    cv_results, n_estimators = cat_cv(cat, X_train, y_train, categorical_features_pos)
    run_time = timer() - start
    # Extract the best score
    best_score = 1 - cv_results

    # Loss must be minimized
    loss = cv_results

    # Boosting rounds that returned the highest cv score


    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'iterations': n_estimators, 'train_time': run_time, 'status': STATUS_OK}



space = {'loss_function':'RMSE',
                           'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
                           'max_depth':hp.quniform('max_depth', 3, 10, 1),
                           'eval_metric':'RMSE',
                           'bootstrap_type':'Bernoulli',
                           'subsample':0.5,
                           'l2_leaf_reg':hp.uniform('l2_leaf_reg', 0.0, 1.0),
                           'random_seed':42,
                            'od_type':'IncToDec',
                           'od_pval':0.01,
                           'max_ctr_complexity':1,
                           'random_strength':hp.uniform('random_strength', 0.0, 1),
                           'leaf_estimation_method':'Newton'
                           
                           }


def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]
categorical_features_pos = column_index(X_train,cat_features)



# File to save first results
out_file = 'cat_new_feat_ap.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()


In [ ]:
from hyperopt import Trials
ITERATION = 0
bayes_trials = Trials()
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest,
            max_evals = 30, trials = bayes_trials, rstate =np.random.RandomState(42))

  0%|          | 0/30 [00:00<?, ?it/s, best loss: ?]